## Statistiques descriptives de la base contenant les informations sur les incidents par arme à feu aux Etats-Unis de 2013 à 2018.

### Statistiques concernant les caractéristiques des incidents étant survenus

Import des packages:

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)
import plotly.express as px

Import de la base de données :

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
updated_base=pd.read_csv("updated_base.csv")
updated_base.head()

In [ ]:
updated_base.rename(columns={'Unnamed: 0':'index'}, inplace=True)

In [ ]:
updated_base

### Statistiques générales

#### Présentation de la base de données 

In [ ]:
print(f" La base de données comporte des renseignements sur {updated_base.count()['incident_id']} incidents par arme à feu entre 2013 et 2018.\n")

#### Statistiques concernant la date des incidents

In [ ]:
print("I. Comment prédire la fréquence des incidents en fonction de leur date ?\n")

In [ ]:
#changing the format of the date
updated_base['date']=pd.to_datetime(updated_base['date'])
updated_base['year_month'] =updated_base['date'].dt.year.astype(str)+"-"+updated_base['date'].dt.month.astype(str)

In [ ]:
fig1 = px.bar(updated_base, x=pd.unique(updated_base['date'].dt.year), y=updated_base.groupby(updated_base['date'].dt.year).count()['incident_id'],
              labels={"x":"date", "y":"nombre d'incidents"},
              title="Nombre d'incidents par arme à feu par an")


fig1.show()

In [ ]:
print("Ce graphique nous montre que les données concernant les années 2013 et 2018 ne sont pas complètes, probablement en raison de la collecte des données. \nMalgré tout, ces données nous permettent d'illustrer une tendance à la hausse du nombre d'incidents par an puisqu'ils passent d'environ 52 000 en 2014 à 61 500 en 2017. ")

In [ ]:
filtered_death=updated_base[updated_base['n_killed']>=1]
filtered_death

fig2 = px.bar(filtered_death[filtered_death['date'].dt.year.isin([2014,2015,2016,2017])], x=pd.unique(filtered_death[filtered_death['date'].dt.year.isin([2014,2015,2016,2017])]['date'].dt.year), y=filtered_death[filtered_death['date'].dt.year.isin([2014,2015,2016,2017])].groupby(filtered_death['date'].dt.year).count()['incident_id'],
              labels={"x":"date", "y":"nombre d'incidents provoquant au moins un mort"},
              title="Nombre d'incidents mortels par arme à feu par an")


fig2.show()

In [ ]:
print("Ce graphique nous permet d'illuster la hausse des incidents mortels par arme à feu aux Etats-Unis depuis le début des années 2010 décrite par le Centers for Disease Control and Prevention (CDC).")

In [ ]:
fig3 = px.bar(updated_base, x=pd.unique(updated_base['year_month']), 
            y=updated_base.groupby([updated_base['date'].dt.year, updated_base['date'].dt.month]).count()['incident_id'],
              labels={"x":"date", "y":"nombre d'incidents"},
              title="Nombre d'incidents par arme à feu par mois")


fig3.show()

In [ ]:
fig4 = px.bar(updated_base[updated_base['date'].dt.year.isin([2014,2015,2016,2017])], 
            x=pd.unique(updated_base[updated_base['date'].dt.year.isin([2014,2015,2016,2017])]['date'].dt.month_name()), 
            y=updated_base[updated_base['date'].dt.year.isin([2014,2015,2016,2017])].groupby(updated_base[updated_base['date'].dt.year.isin([2014,2015,2016,2017])]['date'].dt.month).count()['incident_id'],
            labels={"x":"date", "y":"nombre d'incidents"},
            title="Nombre d'incidents par mois (cumulé de 2014 à 2017)")


fig4.show()


In [ ]:
print("Ces deux graphiques décrivant le nombre d'incident par mois, chaque année ou en cumulé, permettent d'observer une légère saisonnalité des incidents. \nEn effet, ils sont plus nombreux les mois d'été (Juillet et Août) et moins nombreux en Février. \nCela est cohérent avec la littérature sur le sujet (Vivian H. Lyons, Emma L. Gause, Keith R. Spangler, Gregory A. Wellenius, Jonathan Jay. Analysis of Daily Ambient Temperature and Firearm Violence in 100 US Cities. JAMA Network Open, 2022; 5 (12): e2247207 DOI: 10.1001/jamanetworkopen.2022.47207).")

#### Statistiques concernant la localisation des incidents

In [ ]:
print("\nII. La localisation géographique peut-elle influer le nombre d'incidents par arme à feu aux Etats-Unis ?\n")

In [ ]:
fig5 = px.bar(updated_base, x=sorted(pd.unique(updated_base['state'])), 
    y=updated_base.groupby('state').count()['incident_id'], title="Nombre d'incidents par arme à feu par Etat",
    labels={"x":"Etat", "y":"nombre d'incidents"})

fig5.show()

In [ ]:
print("On remarque ici que les Etats dans lesquels le plus d'incidents par arme à feu ont lieu sont : l'Illinois (17.5k), la Californie (16.3k), la Floride (15k) et le Texas (13.5k). \nAu vu de la variation significative entre les différents Etats, on peut en déduire qu'il va falloir aller étudier les caractéristiques de ces Etats, voire des comtés dans lesquels les incidents ont eu lieu, pour identifier des facteurs d'explications.")

#### Caractéristiques des incidents par arme à feu

In [ ]:
print("\nIII. Quelles sont les caractéristiques récurrentes des incidents par arme à feu ?\n")

In [ ]:
fig7 = px.bar(updated_base, x=sorted(pd.unique(updated_base['n_killed'])), y=updated_base.groupby(updated_base['n_killed']).count()['incident_id'],
              labels={"x":"nombre de morts", "y":"nombre d'incidents (échelle logarithmique)"},
              title="Nombre de morts par incident",
              log_y=True)

fig7.show()

In [ ]:
print("On voit ici que la majorité des incidents n'entraînent pas de mort (185.5k, soit 77% des incidents renseignés ici) et que près de 50.000 incidents provoquent une mort. \nQuelques outliers provoquent au delà de 10 morts, ils correspondent aux tueries dites de masse dont voici les références ci-dessous.")

In [ ]:
mass_kill=updated_base[updated_base['n_killed']>=10]
print(mass_kill[['date','state','city_or_county','source_url']])

In [ ]:
fig8 = px.bar(updated_base, x=sorted(pd.unique(updated_base['n_injured'])), y=updated_base.groupby(updated_base['n_injured']).count()['incident_id'],
              labels={"x":"nombre de blessés", "y":"nombre d'incidents (échelle logarithmique)"},
              title="Nombre de blessés par incident",
              log_y=True)

fig8.show()

In [ ]:
print("Le nombre de blessés par incidents suit globalement la même distribution que le nombre de morts par incident mais est plus étalée. \nDans 142.5k cas il n'y a aucun blessé, mais dans 81.000 incidents il y a un blessé. \nLes incidents provoquant plus de 10 blessés sont rares, les références sont disponibles ci-dessous. On y retrouve les incidents très meurtriers (Orlando, Sutherland Springs) mais également d'autres n'ayant pas fait beaucoup de morts.")

In [ ]:
mass_injured=updated_base[updated_base['n_injured']>10]
print(mass_injured[['date','state','city_or_county','source_url']])

In [ ]:
print("Les données liées aux caractéristiques (variable 'incident_characteristics') sont exprimés en fonction de plus d'une trentaine de labels différents, intéressons nous donc d'abord aux descriptions comportant la mention de suicides car c'est, selon le Centers for Disease Control and Prevention's (CDC) National Center for Health Statistics, la cause principale de mort lors des incidents par arme à feu.")

In [ ]:
fig9 = px.bar(updated_base[updated_base['n_killed']>=1], x=pd.unique(updated_base['indic_suicide']), 
            y=updated_base[updated_base['n_killed']>=1].groupby(updated_base['indic_suicide']).count()['incident_id'],
              labels={"x":"incident impliquant un suicide (1) ou non (0)", "y":"nombre d'incidents"},
              title="Parmi les incidents impliquant au moins un mort, combien sont le résultat d'un suicide ?")

fig9.show()

In [ ]:
print("Parmi les incidents impliquant au moins un mort, la majorité (55%) concernent un suicide. On retrouve donc bien les conclusions du CDC sur la question.")

In [ ]:
fig10 = px.bar(updated_base, x=pd.unique(updated_base['indic_accident']), 
            y=updated_base.groupby(updated_base['indic_accident']).count()['incident_id'],
              labels={"x":"incident impliquant un accident (1) ou non (0)", "y":"nombre d'incidents"},
              title="Nombre d'incidents par arme à feu résultant d'un accident")

fig10.show()

In [ ]:
print("Environ 36 000 incidents, soit 15% de l'échantillon total, résultent d'un accident. Cela semble donc être une circonstance récurrente. On pourra donc s'intéresser aux conditions qui rendent possible un accident (et notamment le simple fait de posséder une arme). ")

In [ ]:
fig11 = px.bar(updated_base, x=pd.unique(updated_base['indic_home_inv']), 
            y=updated_base.groupby(updated_base['indic_home_inv']).count()['incident_id'],
              labels={"x":"incident impliquant une violation de domicile ('home invasion') (1) ou non (0)", "y":"nombre d'incidents"},
              title="Nombre d'incidents par arme à feu résultant d'une violation de domicile")

fig11.show()

In [ ]:
print("Le nombre d'incidents par arme à feu résultant d'une violation de domicile ('home invasion') est de 10 434, soit environ 4,5% de l'échantillon. Ainsi c'est une situation moins importante mais tout de même significative.")

In [ ]:
fig12 = px.bar(updated_base, x=pd.unique(updated_base['indic_defens']), 
            y=updated_base.groupby(updated_base['indic_defens']).count()['incident_id'],
              labels={"x":"incident impliquant une défense (1) ou non (0)", "y":"nombre d'incidents"},
              title="Nombre d'incidents impliquant un acte de défense")

fig12.show()

In [ ]:
print("Les incidents impliquant un acte de défense, d'auto-défense, sont au nombre de 7266 dans notre base de données, sachant que ces incidents se recoupent potentiellement avec les incidents qui concernent une violation de domicile.")

In [ ]:
fig13 = px.bar(updated_base, x=pd.unique(updated_base['indic_school']), 
            y=updated_base.groupby(updated_base['indic_school']).count()['incident_id'],
              labels={"x":"incident dans le cadre de l'école (1) ou non (0)", "y":"nombre d'incidents"},
              title="Nombre d'incidents se déroulant dans un cadre scolaire")

fig13.show()

In [ ]:
print("Les incidents par arme à feu ayant lieu dans un cadre scolaire sont au nombre de 2778 (1,15%).")

Pour faire le graphique suivant, il est nécessaire d'importer la base de données df_suspects dans lequel est extraite l'information sur la relation du suspect aux victimes, qui permet également d'expliquer les circonstances de l'incident.

In [ ]:
df_suspects=pd.read_csv('df_suspects.csv')

In [ ]:
filtered_suspects_relations = df_suspects[pd.notna(df_suspects['relation'])]

fig14 = px.bar(filtered_suspects_relations, x=sorted(pd.unique(filtered_suspects_relations['relation'])), 
            y=filtered_suspects_relations.groupby('relation').count()['id'],
              labels={"x":"type de relation entre le suspect et la/les victimes", "y":"nombre de relations concernés"},
              title="Nombre d'incidents dans lesquels ces relations/situations ont joué un rôle",
              color=filtered_suspects_relations.groupby('relation').count()['id'], color_continuous_scale='Speed')

fig14.show()

In [ ]:
fig15 = px.bar(filtered_suspects_relations, x=sorted(pd.unique(filtered_suspects_relations['relation'])), y=filtered_suspects_relations.groupby('relation').count()['id'],
              labels={"x":"type de relation entre le suspect et la/les victimes", "y":"nombre de relations concernés"},
              title="Nombre d'incidents dans lesquels ces relations/situations ont joué un rôle",
              log_y=True,
              color=filtered_suspects_relations.groupby('relation').count()['id'], color_continuous_scale='Speed')

fig15.show()

In [ ]:
print(f"Ces figures, l'une en échelle linéaire et l'autre en échelle logarithmique pour distinguer les faibles valeurs, nous donnent un apperçu plus précis des circonstances récurrentes des incidents par arme à feu. \nLa variable utilisée ici est un dictionnaire donnant une correspondance entre chaque suspect et sa relation aux autres 'participants' à l'incident, elle ne compte donc pas exactement les incidents concernés par ce type de relations mais le nombre de suspects concernés. \nLes données ne sont disponibles que pour {filtered_suspects_relations.shape[0]} suspects sur les {df_suspects.shape[0]} présents dans la base originelle donc les informations sont à prendre avec précaution car malgré le fait qu'elles restent nombreuses, le renseignement de l'information pourrait être biaisé. \nLa situation la plus fréquente est celle des vols à main armée (39% des exprimés), suivie par la famille (17%) et le ou la partenaire ou ex-partenaire (17% également). \nAinsi, environ 39% des incidents sont causés par un suspect proche des victimes (famille, partenaire, ami) et au moins 9% par une connaissance des victimes ('acquaintance', collègue, voisin). \nAu contraire, très peu d'incidents concernent des suspects coupables de fusillade de masse (13 relations entre suspect et victime inconnue et 13 relations entre suspect et victime connue).")

In [ ]:
filtered_suspects_gun = df_suspects[pd.notna(df_suspects['gun_type'])]
filtered_suspects_gun=filtered_suspects_gun[~filtered_suspects_gun['gun_type'].isin(['Unknown'])]

fig16 = px.bar(filtered_suspects_gun, x=sorted(pd.unique(filtered_suspects_gun['gun_type'])), y=filtered_suspects_gun.groupby('gun_type').count()['id'],
              labels={"x":"arme utilisée", "y":"nombre de suspects concernés"},
              title="Nombre de suspect concernés par un incident avec ces armes",
              log_y=True,
              color=filtered_suspects_gun.groupby('gun_type').count()['id'],color_continuous_scale='Speed')


fig16.show()

In [ ]:
print(f"L'arme la plus utilisée lors de ces incidents, lorsqu'elle est renseignée ({filtered_suspects_gun.shape[0]} sur {df_suspects.shape[0]} suspects), est l'arme de poing (40%) suivie par le 9mm (12%). Les carabines ('rifle') et fusils à pompe ('shotgun') sont également très utilisés (6% chacun). \nCes armes qui sont les plus présentes lors d'incidents sont aussi celles qui sont les plus répandues aux Etats-Unis d'après les données du Bureau of Alcohol, Tobacco, Firearms and Explosives (présentées dans cet article : https://www.cbsnews.com/pictures/most-popular-guns-in-america/) puisque les 'Handguns' sont les plus populaires, suivis des 'Rifles' puis des 'Shotgun'. \nAinsi, le fait qu'une arme soit répandue dans la population augmente sa probabilité d'être utilisée lors d'un incident, il faudra donc également s'intéresser aux législations fédérales qui font que certains types d'armes sont plus facilement disponibles que d'autres.")

Création d'une base merge pour récupérer l'information sur les blessés dans la même base de données que celle sur les suspects.

In [ ]:
merge_sus=pd.merge(updated_base, df_suspects, how='outer', left_on='index', right_on='id')

### Caractéristiques des suspects 

In [ ]:
print("\nIV. Quelles sont les caractéristiques des suspects qui sont à l'origine de ces violences par arme à feu ?\n")

In [ ]:
filtered_suspects_gender=df_suspects[pd.notna(df_suspects['gender_sus'])]

fig18 = px.bar(filtered_suspects_gender, x=sorted(pd.unique(filtered_suspects_gender['gender_sus'])), y=filtered_suspects_gender.groupby('gender_sus').count()['id'],
              labels={"x":"genre du suspect", "y":"nombre de suspects"},
              title="Nombre de suspect par genre")

fig18.show()

In [ ]:
print("Les suspects des incidents par arme à feu sont en très grande majorité des hommes (93% de l'échantillon).")

In [ ]:
age=list(range(5,100)) 

filtered_suspects_age=df_suspects[df_suspects['age_sus'].isin(age)]

fig19 = px.bar(filtered_suspects_age, x=sorted(pd.unique(filtered_suspects_age['age_sus'])), y=filtered_suspects_age.groupby('age_sus').count()['id'],
            labels={"x":"âge du suspect", "y":"nombre de suspects"},
            title="Nombre de suspect par tranche d'âge",
            color=filtered_suspects_age.groupby('age_sus').count()['id'], color_continuous_scale='Speed')

fig19

In [ ]:
fig19bis=px.box(filtered_suspects_age, y=filtered_suspects_age['age_sus'], 
    title="Boxplot de l'âge des suspects")
fig19bis.show()

In [ ]:
print("Ce graphique nous montre que la distribution des âges des suspects est très fortement concentrée autour de 18 à 25 ans, la majorité des suspects sont donc des hommes relativement jeunes. ")
print("De plus, le boxplot nous indique que la médiane de l'âge des suspects est de 26 ans et que tous les individus ayant plus de 57 ans sont des 'outliers'.")

In [ ]:
filtered_suspects_status=df_suspects[pd.notna(df_suspects['status_sus'])]

fig20 = px.bar(filtered_suspects_status, x=sorted(pd.unique(filtered_suspects_status['status_sus'])), 
            y=filtered_suspects_status.groupby('status_sus').count()['id'],
              labels={"x":"statut du suspect", "y":"nombre de suspects (échelle logarithmique)"},
              title="Statut des suspects à l'issue de l'incident",
              log_y=True,
              color=filtered_suspects_status.groupby('status_sus').count()['id'],
              color_continuous_scale='Speed')
fig20.show()

In [ ]:
print(f"Ce graphique nous montre qu'à l'issue de la majorité des incidents par arme à feu, le ou les suspect(s) n'est (ne sont) pas blessé(s) (80k 'Unharmed' et 84k 'Unharmed, Arrested'). \nDans cette base de données, on peut remarquer qu'une forte proportion des suspects sont arrêtés ({int(100*84742/filtered_suspects_status.shape[0])}% sont 'Unharmed, Arrested', {int(100*10099/filtered_suspects_status.shape[0])}% sont 'Arrested', {int(100*2850/filtered_suspects_status.shape[0])}% sont 'Injured, Arrested' et une cinquantaine d'individus sont également déclarés 'Arrested' mais dans des catégories négligeables). \nOn remarque également que {int(100*8988/filtered_suspects_status.shape[0])}% des suspects sont déclarés 'Killed', ce qui est significatif. ")

#### Statistiques concernant les caractéristiques des victimes de ces incidents 

In [ ]:
print("\nV. Qui sont les victimes de ces violences par arme à feu ?\n")

In [ ]:
df_victim=pd.read_csv("df_victim.csv")

In [ ]:
filtered_victim_gender=df_victim[pd.notna(df_victim['gender_vict'])]

fig21 = px.bar(filtered_victim_gender, x=sorted(pd.unique(filtered_victim_gender['gender_vict'])), y=filtered_victim_gender.groupby('gender_vict').count()['id'],
              labels={"x":"genre de la victime", "y":"nombre de victimes"},
              title="Genre des victimes")

fig21.show()

In [ ]:
print(f"Ce graphique nous montre qu'au niveau des victimes aussi les hommes sont majoritaires, puisqu'ils représentent {int(100*138615/filtered_victim_gender.shape[0])}% des victimes présentes dans notre base. C'est néanmoins une proportion légèrement plus faible que celle des hommes parmi les suspects.")

In [ ]:
age=list(range(100))

filtered_vict_age = df_victim[df_victim['age_vict'].isin(age)].copy()  

fig22 = px.bar(filtered_vict_age, x=sorted(pd.unique(filtered_vict_age['age_vict'])), y=filtered_vict_age.groupby('age_vict').count()['id'],
            labels={"x":"âge de la victime", "y":"nombre de victimes"},
            title="Nombre de victimes par âge", 
            color=filtered_vict_age.groupby('age_vict').count()['id'], color_continuous_scale='Speed')

fig22.show()

In [ ]:
fig22bis=px.box(filtered_vict_age, y=filtered_vict_age['age_vict'], 
    title="Boxplot de l'âge des victimes")
fig22bis.show()

In [ ]:
print("La distribution des âges des victimes est relativement similaire à celle des âges des suspects (le premier quantile est de 20 ans et le troisième de 36 ans, comme pour les suspects) mais semble légèrement plus étalée. Les âges les plus représentés sont également entre 18 et 25 ans mais il y a une présence non-négligeable de très jeunes victimes (moins de 14 ans).")

In [ ]:
filtered_vict_status=df_victim[pd.notna(df_victim['status_vict'])]

fig23 = px.bar(filtered_vict_status, x=sorted(pd.unique(filtered_vict_status['status_vict'])), y=filtered_vict_status.groupby('status_vict').count()['id'],
              labels={"x":"statut de la victime", "y":"nombre de victimes"},
              title="Statut des victimes", 
            color=filtered_vict_age.groupby('status_vict').count()['id'], color_continuous_scale='Speed')

fig23.show()

In [ ]:
print(f"On remarque sur ce graphique que les victimes n'ont que 3 statuts significatifs à l'issue des incidents par arme à feu. {int(100*110004/filtered_vict_status.shape[0])}% des victimes de notre base de donnée sont blessées, {int(100*51393/filtered_vict_status.shape[0])}% sont tuées et seulement {int(100*24400/filtered_vict_status.shape[0])}% ne sont pas blessées. \nIl faut tout de même prendre en compte que {filtered_vict_status.shape[0]} status de victimes sont déclarés pour {updated_base.shape[0]} incidents. Ainsi, plus de 52 271 incidents n'ont pas fait de victimes (ce chiffre est la fourchette basse étant donné que plusieurs victimes peuvent être déclarées pour un même incident). ")